###**Hedging Strategy Implementation**

####Rolling Beta Calculation

In [ ]:
d = 45  # Set the rolling window size (days)
beta_list = []  # Initialize a list to store calculated beta values

# Loop through the index returns to calculate beta for each time point
for i in range(len(index_returns[:start_date]) - 1, len(index_returns)):
    end_date2 = index_returns.iloc[i].name  # Get the current end date

    start_date2 = end_date2 - dt.timedelta(days=d)  # Define the start date for the rolling window
    model = LinearRegression()  # Initialize the Linear Regression model

    # Prepare the feature (index returns) and target (portfolio returns) for regression
    X = index_returns[start_date2:end_date2].values.reshape(-1, 1)  # Index returns as predictor
    y = port_ret[start_date2:end_date2].values  # Portfolio returns as response

    model.fit(X, y)  # Fit the model to the data
    beta = model.coef_[0]  # Extract the calculated beta coefficient
    beta_list.append(beta)  # Append the beta value to the list

####Number of Contracts, Portfolio & Futures Values Functions

In [ ]:
# Function to calculate the number of futures contracts needed for hedging
def n_contracts(value, index_price, beta, multiplier):
    return -beta * (value) / (index_price * multiplier)  # Calculate the required number of contracts

# Function to calculate the total portfolio value based on quantities and historical prices
def port_v(port_quantity, historical):
    return sum(port_quantity * historical)  # Compute the total portfolio value

# Function to calculate the value of futures contracts based on the number of contracts and historical prices
def index_v(number_of_contracts, historical, multiplier):
    return number_of_contracts * historical.iloc[0] * multiplier  # Calculate the futures contract value

####Calculate Futures Contracts and Portfolio Values

In [ ]:
count = 0  # Initialize a counter for iterations
port_value = []  # List to store portfolio values over time
index_value = []  # List to store index values over time

new_port_equity = port_equity  # Use the existing portfolio equity for calculations
new_index_prices = index_prices[start_date:end_date]  # Slice index prices for the relevant period

# Loop through the index prices to calculate futures contracts and values
for i in range(0, len(new_index_prices)):
    if i == 0:  # For the first iteration
        count += 1  # Increment the counter
        # Calculate the number of contracts needed at the initial index price
        N = [n_contracts(equity, new_index_prices.iloc[0], beta_list[i], multiplier).iloc[0]]
        port_value.append(equity)  # Store the initial portfolio equity
        index_value.append(abs(index_v(N[i], new_index_prices.iloc[0], multiplier)))  # Store the index value
    elif i % frequency == 0:  # Every specified frequency
        count += 1  # Increment the counter
        port_value.append(new_port_equity.iloc[i])  # Store the portfolio value at the current index
        # Calculate the number of contracts needed based on the current portfolio value
        N.append(n_contracts(port_value[i], new_index_prices.iloc[i], beta_list[i], multiplier).iloc[0])
        index_value.append(abs(index_v(N[i], new_index_prices.iloc[i], multiplier)))  # Store the index value
    else:  # For all other iterations
        count += 1  # Increment the counter
        N.append(N[i - 1])  # Retain the previous number of contracts
        port_value.append(new_port_equity.iloc[i])  # Store the portfolio value
        index_value.append(abs(index_v(N[i], new_index_prices.iloc[i], multiplier)))  # Store the index value

####P&L, Required Margin, Margin & Margin Flows

In [ ]:
pnl = [0]  # Profit and Loss (P&L) list, starting with zero
req_margin = list(0.1 * np.array(index_value))  # Required margin, set as 10% of index values
margin = [req_margin[0]]  # Initial margin based on the first required margin value
margin_flows = [-req_margin[0]]  # Initial margin flows (outflow for the first period)
total_equity = [port_equity[0]]  # Starting total equity based on the initial portfolio value

t_equity_change = [np.nan]  # Total equity change (percentage), starting with NaN for the first entry

# Loop through index prices to calculate financial metrics over time
for i in range(1, len(new_index_prices)):
    # Calculate the profit and loss for the current period based on the change in index price
    pnl.append(float((new_index_prices.iloc[i] - new_index_prices.iloc[i - 1]) * 2 * N[i - 1]))

    # Check if the current required margin exceeds the sum of previous margin and current P&L
    if float(req_margin[i]) > float((margin[i - 1] + pnl[i])):
        # If it does, calculate the margin flow (outflow) needed to meet the required margin
        margin_flows.append(-(req_margin[i] - (margin[i - 1] + pnl[i])))
    else:
        margin_flows.append(0)  # No margin flow needed if the margin is sufficient

    # Calculate the total equity, which is the sum of portfolio equity and current P&L
    total_equity.append(port_equity[i] + pnl[i])

    # Calculate the percentage change in total equity compared to the previous period
    t_equity_change.append((total_equity[i] - total_equity[i - 1]) / total_equity[i - 1])

    # Update the required margin by adding the current P&L to the previous required margin
    req_margin.append(req_margin[i] + pnl[i])

    # Update the margin by adjusting for P&L and any margin flows
    margin.append(margin[i - 1] + pnl[i] - margin_flows[i])

# Trim the required margin list to match the length of new index prices
req_margin = req_margin[:len(new_index_prices)]


<ipython-input-21-a33bd3112fa1>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_equity = [port_equity[0]]  # Starting total equity based on the initial portfolio value
<ipython-input-21-a33bd3112fa1>:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pnl.append(float((new_index_prices.iloc[i] - new_index_prices.iloc[i - 1]) * 2 * N[i - 1]))
<ipython-input-21-a33bd3112fa1>:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_equity.append(port_equity[i] + pnl[i])


####Final Results Table

In [ ]:
# Prepare results into a DataFrame for final output
final_result = pd.concat([
    pd.DataFrame(new_index_prices.index),  # Date index for the new index prices
    pd.DataFrame(beta_list),               # List of calculated beta values
    pd.DataFrame(N),                       # Number of futures contracts for hedging
    pd.DataFrame(index_value),             # Values of the futures contracts
    pd.DataFrame(new_index_prices.values), # Closing prices of the index
    pd.DataFrame(pnl),                     # Profit and Loss values
    pd.DataFrame(margin),                  # Margin values throughout the period
    pd.DataFrame(req_margin),               # Required margin values
    pd.DataFrame(margin_flows),            # Flows of margin (in/out) during the period
    pd.DataFrame(port_equity.reset_index(drop=True)), # Portfolio value, reset index to match lengths
    pd.DataFrame(total_equity),            # Total equity values after accounting for P&L
    pd.DataFrame(t_equity_change)          # Percentage changes in total equity
], axis=1)  # Concatenate along columns

# Assign meaningful column names to the final DataFrame
final_result.columns = [
    'Date',                       # Date of the entry
    'beta',                       # Rolling beta values for hedging
    'N',                          # Number of futures contracts
    'FTSE_value',                # Value of the FTSE contracts
    'FTSE Close',                # Closing price of the FTSE index
    'P/L',                       # Profit and Loss for the period
    'Margin',                    # Margin maintained
    'Required Margin',           # Required margin as per the contracts
    'Margin Flows',              # Margin flows (in or out)
    'Portfolio Value',           # Value of the portfolio
    'Hedged Portfolio Value',    # Value of the portfolio after hedging
    'Hedged Portfolio Returns'    # Returns on the hedged portfolio
]

# Set the 'Date' column as the index for the final result DataFrame
final_result.index = final_result['Date']
